In [14]:
import numpy as np
import math
import os.path
import csv
import glob
import tensorflow as tf
import h5py as h5py
from keras.preprocessing import image
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model, load_model

In [15]:
# !pip install imageio
# !pip install mlxtend
# !pip install dlib
!pip install scikit-image
import imageio
import matplotlib.pyplot as plt
from mlxtend.image import extract_face_landmarks

/bin/bash: /home/vivek/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
! ls '/content/drive/My Drive/'

 Fold5_part1   Fold5_part2  'Getting started.pdf'


In [16]:
def eye_aspect_ratio(eye):
	A = distance.euclidean(eye[1], eye[5])
	B = distance.euclidean(eye[2], eye[4])
	C = distance.euclidean(eye[0], eye[3])
	ear = (A + B) / (2.0 * C)
	return ear

In [17]:
def mouth_aspect_ratio(mouth):
    A = distance.euclidean(mouth[14], mouth[18])
    C = distance.euclidean(mouth[12], mouth[16])
    mar = (A ) / (C)
    return mar

In [18]:
def circularity(eye):
    A = distance.euclidean(eye[1], eye[4])
    radius  = A/2.0
    Area = math.pi * (radius ** 2)
    p = 0
    p += distance.euclidean(eye[0], eye[1])
    p += distance.euclidean(eye[1], eye[2])
    p += distance.euclidean(eye[2], eye[3])
    p += distance.euclidean(eye[3], eye[4])
    p += distance.euclidean(eye[4], eye[5])
    p += distance.euclidean(eye[5], eye[0])
    return 4 * math.pi * Area /(p**2)

In [19]:
def mouth_over_eye(eye):
    ear = eye_aspect_ratio(eye)
    mar = mouth_aspect_ratio(eye)
    mouth_eye = mar/ear
    return mouth_eye

In [48]:
def getFrame(sec):
    start = 180000
    print(vidcap.set(cv2.CAP_PROP_POS_MSEC, start + sec*1000))
    hasFrames,image = vidcap.read()
    return hasFrames, image

In [73]:
# main program
from scipy.spatial import distance
from numpy import asarray
from PIL import Image
import cv2
data = []
labels = []

for i in [0,5,10]:
    folder = '../Dataset/Train/'+str(f)
    for filename in os.listdir(folder):
            print(filename)
            image = cv2.imread(os.path.join(folder,filename))
            landmarks = extract_face_landmarks(image)
            if  landmarks is not None :
              if sum(sum(landmarks)) != 0:
                  count += 1
                  data.append(landmarks)
                  print('label',i)
                  labels.append([i])
                  sec = sec + frameRate
                  print(sec)
                  sec = round(sec, 2)
                  success, image = getFrame(sec)
                  print(count)
              else:  
                  sec = sec + frameRate
                  sec = round(sec, 2)
                  success, image = getFrame(sec)
                  print("not detected")

len(data)
len(labels)

In [62]:
len(data[0])

68

In [75]:
import numpy as np
data = np.array(data)
labels = np.array(labels)

In [76]:
data

array([[[558, 270],
        [560, 296],
        [563, 321],
        ...,
        [664, 353],
        [654, 354],
        [645, 352]],

       [[559, 281],
        [561, 308],
        [564, 333],
        ...,
        [665, 371],
        [656, 373],
        [648, 373]],

       [[262, 761],
        [258, 809],
        [257, 857],
        ...,
        [519, 901],
        [501, 902],
        [482, 901]],

       ...,

       [[535, 339],
        [539, 363],
        [543, 387],
        ...,
        [677, 383],
        [668, 383],
        [658, 381]],

       [[523, 339],
        [526, 361],
        [530, 383],
        ...,
        [646, 376],
        [636, 376],
        [626, 375]],

       [[555, 271],
        [557, 296],
        [559, 320],
        ...,
        [659, 341],
        [650, 342],
        [642, 341]]], dtype=int32)

In [101]:
features = []
for d in data:
  eye = d[36:68]
  ear = eye_aspect_ratio(eye)
  mar = mouth_aspect_ratio(eye)
  cir = circularity(eye)
  mouth_eye = mouth_over_eye(eye)
  features.append([ear, mar, cir, mouth_eye])

In [104]:
features = np.array(features)
features.shape

(1707, 4)

In [105]:
data.shape

(1707, 68, 2)

In [106]:
labels

array([[ 0],
       [ 0],
       [ 0],
       ...,
       [10],
       [10],
       [10]])

In [110]:
np.save(open('train_features.npy','wb'), features)
np.save(open('train_features_labels.npy', 'wb'),labels)
np.savetxt('train_features.csv', features, delimiter = ",", header="ear, mar, cir, mouth_eye",comments="")
np.savetxt('train_features_labels.csv',labels, delimiter = ",", header="Y",comments="")

In [90]:
features.shape, labels.shape

((1707, 4), (1707, 1))

In [2]:
import pandas as pd
df = pd.read_csv('train_features.csv')
df.head()

,ear,mar,cir,mouth_eye
0,0.259239,0.990172,0.377137,3.819529
1,0.287879,0.904440,0.447990,3.141738
2,0.179442,0.452492,0.334622,2.521654
3,0.289862,0.518949,0.443054,1.790334
4,0.288585,0.752099,0.442154,2.606156


In [150]:
df.rename(columns = {'ear':'EAR',' mar':'MAR',' cir':'Circulatory', ' mouth_eye':'MOE'}, inplace = True)
df['EAR_N'] = df['EAR'] / df['EAR'].max(axis=0)
df['MAR_N'] = df['MAR'] / df['MAR'].max(axis=0)
df['Circulatory_N'] = df['Circulatory'] / df['Circulatory'].max(axis=0)
df['MOE_N'] = df['MOE'] / df['MOE'].max(axis=0)
df.head()

,EAR,MAR,Circulatory,MOE,EAR_N,MAR_N,Circulatory_N,MOE_N
0,0.259239,0.990172,0.377137,3.819529,0.380600,0.314112,0.387625,0.453853
1,0.287879,0.904440,0.447990,3.141738,0.422647,0.286916,0.460448,0.373315
2,0.179442,0.452492,0.334622,2.521654,0.263447,0.143544,0.343927,0.299634
3,0.289862,0.518949,0.443054,1.790334,0.425558,0.164626,0.455374,0.212735
4,0.288585,0.752099,0.442154,2.606156,0.423684,0.238588,0.454450,0.309675


In [3]:
df1 = pd.read_csv('train_features_labels.csv')
df1

,Y
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
1702,10.0
1703,10.0
1704,10.0
1705,10.0


In [178]:
total = pd.concat([df, df1], axis=1, ignore_index=True)
total.columns = ['EAR', 'MAR', 'Circulatory', 'MOE', 'EAR_N', 'MAR_N', 'Circulatory_N',
       'MOE_N', 'Y']
total.to_csv('totalwithmaininfo1.csv',index=False)

In [161]:
pd.unique(df1['Y'])

array([ 0.,  5., 10.])